# Control flow

## Topics
- Mapping and broadcasting
- The dot-syntax
- More details on the loop syntax
- Array comprehension

See also the [documentation](https://docs.julialang.org/en/v1/manual/control-flow/
)

## Loops

More details on loops and some tips and tricks.

## Enumerate: A life saver
Iterate over an array with index AND value. 

Syntax is
```julia
for (index, value) in enumerate(arr)
    # Loop content
end
```

In [ ]:
# Quick reminder of how for-loops work
for n in 1:10
    println(n)
end

In [ ]:
for (i,s) ∈ enumerate(["foo","bar","baz"])
    println(i, " ", s)
end

## Array comprehension
Comprehensions provide a general and powerful way to construct arrays. 

Comprehension syntax is similar to set construction notation in mathematics
```julia
A = [ F(x,y,...) for x=rx, y=ry, ... ]
```

In [ ]:
[(i,j) for i=1:2 for j=1:i]

## Breaking and continuing
Sometimes you need to terminate a `while` or `for` evaluation before the end.  This can be accomplished with the `break` keyword


In [ ]:
i = 1;
while true
    println(i)
    if i >= 5
        break
    end
    i += 1
end

In [ ]:
for i = 1:1000
    println(i)
    if i >= 5
        break
    end
end

In other circumstances, it is handy to be able to stop an iteration and move on to the next one immediately. The `continue` keyword accomplishes this.


In [ ]:
for i = 1:10
    if i % 3 != 0
        continue
    end
    println(i)
end

## Nested loops
Multiple nested loops can be combined into a single outer loop. 

In [ ]:
for i = 1:2
    for j = 3:3
        println((i,j))
    end
end

Translates into:

In [ ]:
for i = 1:2, j = 3:4
    println((i, j))
end

A `break` statement inside such a loop exits the entire nest of loops, not just the inner one.


## Some higher-order functions: `map`

`map` is a "higher-order" function in Julia that takes a *function* as one of its input arguments. `map` then applies that function to every element of the data structure you pass. 

For example
```julia
map(f, [1,2,3])
```
will correspond to
```julia
[f(1), f(2), f(3)]
```

In [ ]:
function f(x, y)
    return x + y
end

map(f, [1,2,3], [4,5,6])

## Some higher-order functions: `broadcast`
`broadcast` is another higher-order function like `map`. `broadcast` is actually a generalization, so it can do the same as `map` but also more!

Syntax is the same
```julia
broadcast(f, [1,2,3])
```

And so we have again applied f (squared) to all elements of `[1,2,3]`.

In [ ]:
broadcast(f, [1,2,3], [4,5,6])

In [ ]:
broadcast(f, [[4, 5, 6], [1, 2, 3]], [[1,2,3]])

## Broadcasting (or vectorizing)
Some syntactic sugar for calling `broadcast` is to place `.` between the name of the function you want to broadcast and its input arguments. 
        
For example
```julia
broadcast(f, [1,2,3])
```
is the same as
```julia
f.([1,2,3])
```

Note that this is not the same as `f([1,2,3])` because we can not square a vector!

Let's try broadcasting for a matrix `A`

In [ ]:
function f(x)
    return x^2
end

A = [1 2 3;
     4 5 6;
     7 8 9]

In [ ]:
f(A)

In [ ]:
f.(A)

## Dot syntax for vectorization
The dot syntax allows to write complex compound **elementwise** expressions in a way that looks natural/closer to mathematical notation. 

For example:

In [ ]:
A + 2 .* f.(A) ./ A

Instead of the more nasty looking version with `broadcast` as

In [ ]:
broadcast(x-> x + 2 * f(x) / x, A) 

The `.`-syntax is useful and looks nice, if you are used to interpreted languages beware: in Julia a for loop is faster. You don't need to vectorize to get good performance.

### Exercise 1: Map / Broadcast

Define a function called `addone` that adds `1` to any number.

Use `map` or `broadcast` and the `addone` function to increment every element of a matrix `A` by `1`.

You can easily create an empty matrix for testing with `A = zeros(5,5)`.

### Exercise 2: Dot syntax

Try out the broadcast dot syntax to increment every element of matrix `A` by `1` using the function defined in the first exercise.

### Advanced exercise 1: More dots
Generalize the previous polynomial function to work for matrices, too. In practise this means that `x` can be of array
and the polynomial function is evaluated for every element of that array.

## Back to the simulation

Let's get back to the simulation and implement a single time step. But first, we need to create a field filled with plants. We'll use a 2D array to represent the field.

In [ ]:
# Copy necessary things from the previous session

"Enumerate possible states of a single plant"
@enum InfectionStatus uninfected infected dead recovered

"Data structure containing the infection status of a plant"
mutable struct Plant
    status::InfectionStatus
    infection_time::Int8
end

"Update a single plant one time step."
function update!(plant::Plant, recovery_time, death_rate)
    if plant.status == infected
        
        # The plant dies with probability death_rate
        if rand(1)[1] < death_rate
            # Kill the plant
            plant.status = dead
        end
        
        # add 1 to the infection time
        plant.infection_time += 1
        
        # if the infection time is larger than the recovery time, the plant recovers
        if plant.infection_time > recovery_time
            # Kill the plant
            plant.status = recovered
        end
    end
end

"""Simulate an interaction between two plants. In the other plant is
   infected, it may infect the this plant.
"""
function interact!(this_plant::Plant, other_plant::Plant, infection_rate)
    if this_plant.status == uninfected && other_plant.status == infected
        if rand(1)[1] < infection_rate
            this_plant.status = infected
            this_plant.infection_time = 0
        end
    end
end

"Show the plant as an empty or a filled in box for quick viewing"
function Base.show(io::IO, plant::Plant)
    if plant.status == infected
        print(io, "◼")
    else
        print(io, "◻")
    end
end

In [ ]:
# Create a 16 x 16 matrix of plants
plants = Matrix{Plant}(undef, 16, 16)

for i in 1:size(plants)[1]
    for j in 1:size(plants)[2]
        plants[i,j] = Plant(uninfected, 0)
    end
end
plants[8,8].status = infected

Now we can implement a time step in our simulation. For this, we need to loop over each pair of neighbouring plants, since they can infect each other.

First let's check the neighbours in the vertical direction. So if one plant is at (i,j), the plant at (i+1,j) is a neighbour. If a column has N plants, there are N-1 pairs.

In [ ]:
"Update the simulation one time step"
function update!(plants::Matrix{Plant}, recovery_time, death_rate, infection_rate)
    # Create a copy to remember the old state
    old_plants = deepcopy(plants)
    
    # First update individual plants
    update!.(plants, recovery_time, death_rate)
    
    # Find the number of plants in each direction
    Nx = size(plants)[1]
    Ny = size(plants)[2]    
    
    # Loop over pairs of plants in the same row. There are Nx-1 pairs.
    for j in 1:Ny
        # loop over all columns
        for i in 1:Nx-1
            # So the plants are (i+1,j) and (i,j). Each will 
            # interact with the other.
            
            interact!(plants[i,j], old_plants[i+1,j], infection_rate)
            interact!(plants[i+1,j], old_plants[i,j], infection_rate)

        end
    end
end

In [ ]:
update!(plants, 5, 0.02, 0.5)
plants

In [ ]:
plants[7,7].infection_time

### Exercise 3: Spreading vertically

Extend the function above to allow the infection to spread horizontally. The code is repeated below, with a bit of scaffolding.

Check by running the update function repeatedly and showing the plants-array.

In [ ]:
"Update the simulation one time step"
function update!(plants::Matrix{Plant}, recovery_time, death_rate, infection_rate)
    # Create a copy to remember the old state
    old_plants = deepcopy(plants)
    
    # First update individual plants
    update!.(plants, recovery_time, death_rate)
    
    # Find the number of plants in each direction
    Nx = size(plants)[1]
    Ny = size(plants)[2]    
    
    # Loop over pairs of plants in the same row. There are Nx-1 pairs.
    for j in 1:Ny
        # loop over all columns
        for i in 1:Nx-1
            # So the plants are (i+1,j) and (i,j). Each will 
            # interact with the other.
            interact!(plants[i,j], old_plants[i+1,j], infection_rate)
            interact!(plants[i+1,j], old_plants[i,j], infection_rate)
        end
    end
    
    # Loop over pairs of plants in the same column.
    for j in ...
        # loop over all columns
        for i in ...
            # So the plants are (i+1,j) and (i,j). Each will 
            # interact with the other.
            interact!(...)
            interact!(...)
        end
    end
end

### Exercise 4: Printing the array of plants

In [ ]:
# Interestingly, this is a bit different
print(plants)

So let's extend the print function. Add the missing parts below to print the array in a nicer way.

In [ ]:
# This is how we write a show function for an array of plants
function Base.show(io::IO, plants::Array{Plant, 2})
    Nx = size(plants)[1]
    Ny = size(plants)[2]
    
    # Iterate over rows and columns separately
    for j in ...
        for i in ...
            ... 
        end
    print('\n')
    end    
end

print(plants)

# Extra:

## While statement
Syntax is 
```julia
while *condition*
    *loop body*
end
```

In [ ]:
n = 0
while n < 10
    n += 1
    println(n)
end

## Ternary operator
Even though the name for this operation is scary, it is actually very easy to understand and handy to use. Syntax is:
```julia
*condition* ? *do 1* : *do 2*
```
which is equal to writing
```julia
if *condition*
    *do 1*
else
    *do 2*
end
```

In [ ]:
# What does the followning code do? Try it out by giving values to `x` and `y`
x = 
y = 
(x > y ) ? x : y

### Advanced: SIMD vectorization
Vectorization is discussed more in the bonus notebook about SIMD vectorization.

In short, the topic is quite technical but you should rest assured that the dot syntax actually works quite well to make your code easy to read **and** fast to run.